# Opening, navigating and parsing pieces of an ENDF file 
## overview
An ENDF tape contains a lot of information. Older ENDF tapes even contained the data for multiple materials. In most cases, users are only interested in small pieces of the full tape and so it is not very efficient to parse the entire ENDF tape when accessing the file. As a result, `ENDFtk` provides a tree based view into the ENDF file that allows a user to navigate to the specific parts of the ENDF file before parsing the data.

An ENDF tape can be read into this tree structure as follows:

In [ ]:
import ENDFtk

tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pu239.endf' )

An `ENDFtk.tree.Tape` effectively indexes the file into available materials, files and sections, which can be parsed at will by the user. The `ENDFtk.tree` submodule also provides an `ENDFtk.tree.Material`, `ENDFtk.tree.File` and `ENDFtk.tree.Section` that the user can look through before parsing the data that is contained in any of these. Parsing the data requires strict adherence to the ENDF6 format but loading the data into an `ENDFtk.tree.Tape` does not. As a result, the `ENDFtk.tree.Tape` is also capable of indexing older ENDF files that do not conform with the ENDF6 standard (tests have shown that we can index files as old as ENDF/B-II).

Once the data is indexed into an `ENDFtk.tree.Tape` object, a user can navigate to the specific pieces retrieve specific pieces from the tape (entire materials, files or sections) as follows:

In [ ]:
material = tape.material( 9437 )
file = tape.material( 9437 ).file( 3 )
section = tape.material( 9437 ).file( 3 ).section( 18 )

`ENDFtk` also provides an interface that emulates the ENDF nomenclature traditionally used for ENDF files (we will refer to this as 'ENDF speak'). The following piece of code achieves the same results as the previous piece of code, but does so using the traditional ENDF speak:

In [ ]:
material = tape.MAT( 9437 )
file = tape.MAT( 9437 ).MF( 3 )
section = tape.MAT( 9437 ).MF( 3 ).MT( 18 )

A user can also interrogate these objects to see if specific materials, files and sections are present:

In [ ]:
tape.has_material( 9437 )
material.has_file( 3 )
file.has_section( 102 )

This is only one of the ways a user can navigate over a tree. The various components of the tree also allow a user to retrieve all materials, files and sections, which a user can then loop over if desired:

In [ ]:
count = 0
for material in tape.materials :
    
    for file in material.files :
        
        for section in file.sections :
            
            count += 1

At this point, the objects retrieved from the ENDF tree are still unparsed objects. To parse these pieces of the ENDF file, we simply have to call the `parse()` function on these objects:

In [ ]:
material = tape.material( 9437 ).parse()
file = tape.material( 9437 ).file( 3 ).parse()
section = tape.material( 9437 ).file( 3 ).section( 18 ).parse()

Accessing the underlying files and sections from parsed materials and files uses the same function names as the tree components to make the interface as intuitive as possible. In this case, because the parent objects are already parsed, there is no need to parse the resulting object:

In [ ]:
material = tape.material( 9437 ).parse()
file = material.file( 3 )
section = file.section( 18 )

## Application 1: printing an overview of the content of an ENDF tape
With the functionality presented above, we can now develop a simple tool that prints out an overview of the content of an entire ENDF tape along with the total number of lines for each section. The code given below will loop over all materials, files and sections and give a rough overview of the content of an ENDF tape.

In [1]:
import ENDFtk

filename = 'resources/n-Pu239.endf'

tape = ENDFtk.tree.Tape.from_file( filename )
for material in tape.materials :
    
    print( 'Material {:4}'.format( material.material_number ) )
    for file in material.files :
        
        print( '  File {:2}'.format( file.file_number ) )
        for section in file.sections :
            
            print( '    Section {:3}, {:5} lines'.format( section.section_number, section.NC ) )

Material 9437
  File  1
    Section 451,   720 lines
    Section 452,  1059 lines
    Section 455,     7 lines
    Section 456,  1059 lines
    Section 458,  1145 lines
  File  2
    Section 151,  1417 lines
  File  3
    Section   1, 50268 lines
    Section   2, 50268 lines
    Section   4,   142 lines
    Section  16,    45 lines
    Section  17,    23 lines
    Section  18, 50268 lines
    Section  37,     7 lines
    Section  51,   142 lines
    Section  52,   124 lines
    Section  53,   122 lines
    Section  54,   118 lines
    Section  55,   116 lines
    Section  56,   112 lines
    Section  57,   110 lines
    Section  58,   108 lines
    Section  59,   106 lines
    Section  60,   104 lines
    Section  61,   102 lines
    Section  62,   101 lines
    Section  63,   100 lines
    Section  64,    99 lines
    Section  65,    98 lines
    Section  66,    97 lines
    Section  67,    95 lines
    Section  68,    94 lines
    Section  69,    86 lines
    Section  70,    84 lines

## Application 2: parsing all ENDF files in a given directory structure
In the following application, the content of a directory is analysed to retrieve all ENDF files (using the .endf extension) after which all files are fully parsed. This entire process is timed and all failures will be reported.

In [2]:
import os

# a recursive function to find all files with a given extension in a directory
def getFilesInFolder( directory, extension ) :

    files = os.listdir( directory )
    everything = list()

    for entry in files :

        file = os.path.join( directory, entry )

        if os.path.isdir( file ) :

            everything += getFilesInFolder( file, extension )

        else :

            everything.append( file )

    return [ file for file in everything if file.endswith( extension ) ]

In [3]:
import ENDFtk
import time

path = 'resources'
files = getFilesInFolder( path, 'endf' )
failed = []

start = time.perf_counter()
for file in files :

    try :

        tape = ENDFtk.tree.Tape.from_file( file )
        materials = [ material.parse() for material in tape.materials ]

    except :

        print( 'FAILED', file )
        failed.append( file )

end = time.perf_counter()

print( '-------------------------' )
print( '{:4} files'.format( len( files ) ) )
print( '{:4} failed'.format( len( failed ) ) )
print( 'elapsed time = {:.3} s'.format( end - start ) )
print( '-------------------------' )

-------------------------
   7 files
   0 failed
elapsed time = 2.23 s
-------------------------
